In [1]:
# Import Libraries #
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_seq_items = 20

In [2]:
# Import Data #
d_price = pd.read_csv(r'D:\SWP\jupyter_workspace\Interview\BGO\Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
# d_forecast = pd.read_csv(r'D:\SWP\jupyter_workspace\Interview\BGO\Zip_zhvf_growth_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
# d_rent = pd.read_csv(r'D:\SWP\jupyter_workspace\Interview\BGO\Zip_zori_sm_month.csv')
# d_census = pd.read_csv(r'D:\SWP\jupyter_workspace\Interview\BGO\BGO_challenge_census_data.csv')

In [3]:
# Explore
d_price = pd.DataFrame(d_price)

# np.shape(d_price) # 27021 rows and 286 columns
# d_price.head()
# # RegionName = Zipcode
# # Size Rank = 15372 distinct values out of 27021 rows
# # Region Name = Either City or County
# # Region Type = they are all zip
# # Monthly data from 2000.01 ~ 2023.01

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31
0,91940,0,77449,zip,TX,TX,NaN,"Houston-The Woodlands-Sugar Land, TX",Harris County,111025.364504,...,267031.277605,271167.441993,274949.991267,277857.050380,279379.374979,279571.703469,278978.533186,278001.127045,276453.815200,274671.967839
1,91982,1,77494,zip,TX,TX,NaN,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,217709.460879,...,460467.367267,467482.802830,473572.893250,477267.065068,477984.861877,476524.002739,474871.543073,473490.978602,471547.993017,469380.296915
2,93144,2,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,93297.846024,...,180425.647045,182618.423967,184808.073687,186768.821493,188315.027071,189360.332747,190107.287314,190872.067470,191400.577159,192365.842160
3,62080,3,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,75454.474440,...,336639.762806,336736.022093,337304.550369,337577.574287,336818.929821,335031.093053,333107.358734,332435.128586,331124.345954,331082.536088
4,62093,4,11385,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,237548.770794,...,743614.071840,750129.456105,757964.299916,763316.251684,764586.578334,761489.401151,758818.529559,756929.158762,753410.376751,750163.629656


In [73]:
# Only keep January columns. Zillow forecast uses current & last year month.
d_price2 = pd.concat([d_price.iloc[:,0:9], 
                      d_price.loc[:,d_price.columns.str.contains('-01-')]], 
                     axis=1)

print(d_price2.isna().sum())

# drop columns
d_price2 = d_price2.drop(columns=['RegionID', 'RegionType', 'StateName', 'Metro', 'CountyName'])

# print(d_price2.head())
# print(list(d_price2.columns))

RegionID          0
SizeRank          0
RegionName        0
RegionType        0
StateName         0
State             0
City           1353
Metro          6026
CountyName        0
2000-01-31    14091
2001-01-31    13682
2002-01-31    13318
2003-01-31    12829
2004-01-31    12360
2005-01-31    12058
2006-01-31    11680
2007-01-31    11389
2008-01-31    11029
2009-01-31     9972
2010-01-31     8119
2011-01-31     7804
2012-01-31     6252
2013-01-31     5769
2014-01-31     5445
2015-01-31     5085
2016-01-31     4712
2017-01-31     3238
2018-01-31     2927
2019-01-31     2589
2020-01-31     1936
2021-01-31     1839
2022-01-31     1466
2023-01-31       30
dtype: int64


In [37]:
# Transpose
d_price3 = pd.melt(d_price2, 
        id_vars=['RegionName', 'SizeRank', 'State', 'City'], 
        value_vars=['2000-01-31', '2001-01-31', '2002-01-31', '2003-01-31', '2004-01-31', '2005-01-31', '2006-01-31', '2007-01-31', '2008-01-31', '2009-01-31', '2010-01-31', '2011-01-31', '2012-01-31', '2013-01-31', '2014-01-31', '2015-01-31', '2016-01-31', '2017-01-31', '2018-01-31', '2019-01-31', '2020-01-31', '2021-01-31', '2022-01-31', '2023-01-31'],
        var_name='date', 
        value_name='ZHVI'
       )

# convert type to date & keep the year
d_price3["date"] = pd.to_datetime(d_price3["date"])
d_price3["date"] = d_price3["date"].dt.year

# create id for join
d_price3["id"] = d_price3['date'].astype(str) +"_"+ d_price3["RegionName"].astype(str)

print(d_price3.head())
print(d_price3.dtypes)

# print(np.unique(d_price3["RegionName"]))
# print(np.unique(d_price3["SizeRank"]))

   RegionName  SizeRank State      City  date           ZHVI          id
0       77449         0    TX       NaN  2000  111025.364504  2000_77449
1       77494         1    TX       NaN  2000  217709.460879  2000_77494
2       79936         2    TX   El Paso  2000   93297.846024  2000_79936
3       11368         3    NY  New York  2000   75454.474440  2000_11368
4       11385         4    NY  New York  2000  237548.770794  2000_11385
RegionName      int64
SizeRank        int64
State          object
City           object
date            int64
ZHVI          float64
id             object
dtype: object


In [35]:
# Import census
d_census = pd.read_csv(r'D:\SWP\jupyter_workspace\Interview\BGO\BGO_challenge_census_data.csv')
d_census = pd.DataFrame(d_census.sort_index(axis=1))

# create id for join
d_census['id'] = d_census['year'].astype(str) +"_"+ d_census["zcta"].astype(str)

print(d_census.dtypes)
print(d_census.describe)

B01001A_001E       int64
B01001A_017E       int64
B01001_002E        int64
B01001_026E        int64
B01002_001E      float64
B01002_002E      float64
B01002_003E      float64
B01003_001E        int64
B07009_002E      float64
B07009_003E      float64
B07009_004E      float64
B07009_005E      float64
B07009_006E      float64
B07413PR_002E    float64
B07413PR_003E    float64
B08006_017E      float64
B08101_049E      float64
B08537_002E      float64
B08537_003E      float64
B19019_001E      float64
B25013_002E        int64
B25027_010E        int64
B25031_001E      float64
B25032_013E        int64
B25036_002E        int64
B25038_009E        int64
B25042_009E        int64
B25056_001E        int64
B25087_001E        int64
B25087_002E        int64
year               int64
zcta               int64
id                object
dtype: object
<bound method NDFrame.describe of         B01001A_001E  B01001A_017E  B01001_002E  B01001_026E  B01002_001E  \
0              17121          8943         8971   

In [50]:
# join price & census
d_price4 = d_price3.merge(d_census, on='id', how='inner')

# drop columns & sort values
d_price4 = d_price4.drop(columns=['date', 'zcta', 'id', 'SizeRank'])
d_price4 = d_price4.sort_values(by=['RegionName','year'], ascending=True)

print(list(d_price4.columns))
print(d_price4.head())

# check for columns with null values
print(d_price4.isna().any())

['RegionName', 'State', 'City', 'ZHVI', 'B01001A_001E', 'B01001A_017E', 'B01001_002E', 'B01001_026E', 'B01002_001E', 'B01002_002E', 'B01002_003E', 'B01003_001E', 'B07009_002E', 'B07009_003E', 'B07009_004E', 'B07009_005E', 'B07009_006E', 'B07413PR_002E', 'B07413PR_003E', 'B08006_017E', 'B08101_049E', 'B08537_002E', 'B08537_003E', 'B19019_001E', 'B25013_002E', 'B25027_010E', 'B25031_001E', 'B25032_013E', 'B25036_002E', 'B25038_009E', 'B25042_009E', 'B25056_001E', 'B25087_001E', 'B25087_002E', 'year']
        RegionName State    City           ZHVI  B01001A_001E  B01001A_017E  \
6850          1001    MA  Agawam  192217.982081         15996          8357   
33803         1001    MA  Agawam  184880.414560         16227          8433   
60756         1001    MA  Agawam  183377.962960         15910          8361   
87709         1001    MA  Agawam  186557.812542         15839          8409   
114662        1001    MA  Agawam  186071.361308         16014          8397   

        B01001_002E  

In [58]:
# first remove rows with null ZHVI
d_price5 = d_price4.dropna(subset=['ZHVI'])

# check for null values
print(d_price5.isna().sum())
print(len(d_price5.index))

# these columns only contain null. drop them.
#'B07413PR_002E', 'B07413PR_003E', 'B08537_002E', 'B08537_003E'
d_price5 = d_price5.drop(columns=['B07413PR_002E', 'B07413PR_003E', 'B08537_002E', 'B08537_003E'])

# check again
print(d_price5.isna().sum())
print(len(d_price5.index))

RegionName            0
State                 0
City              11372
ZHVI                  0
B01001A_001E          0
B01001A_017E          0
B01001_002E           0
B01001_026E           0
B01002_001E           0
B01002_002E           0
B01002_003E           0
B01003_001E           0
B07009_002E           1
B07009_003E           1
B07009_004E           1
B07009_005E           1
B07009_006E           1
B07413PR_002E    249169
B07413PR_003E    249169
B08006_017E           1
B08101_049E           1
B08537_002E      249169
B08537_003E      249169
B19019_001E        8635
B25013_002E           0
B25027_010E           0
B25031_001E       85592
B25032_013E           0
B25036_002E           0
B25038_009E           0
B25042_009E           0
B25056_001E           0
B25087_001E           0
B25087_002E           0
year                  0
dtype: int64
249169
RegionName          0
State               0
City            11372
ZHVI                0
B01001A_001E        0
B01001A_017E        0
B01001_0

In [71]:
print(np.unique(d_price5[['year']][d_price5['B19019_001E'].isna()]))

print(np.unique(d_price5[['year']][d_price5['B25031_001E'].isna()]))

[2015 2018]
[2011 2012 2013 2014 2015]
